In [1]:
# from IPython.display import clear_output
# !pip install transformers datasets torch evaluate
# clear_output()

In [2]:
# importing libraries 
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

In [3]:
pd.read_csv(r'data\TextClassification\IMDB\train.csv')

,label,text
0,1,Four teenage girls in a suburb of Los Angeles ...
1,1,I was in a bad frame of mind when I first saw ...
2,0,"There is a word for this sort of film, and tha..."
3,0,This was a blind buy used DVD. It totally kill...
4,0,As a youth pastor I heard good things about th...
...,...,...
24995,0,"If you didn't know better, you would believe t..."
24996,0,Franco Rossi's 1985 six-hour Italian mini-seri...
24997,1,An excellent example of the spectacular Busby ...
24998,1,"In the early 1970s, many of us who had embrace..."


In [4]:
# here in this dataset I found two datasets for training and testing purpose seprately
df_train = pd.read_csv(r'data\TextClassification\IMDB\train.csv')
df_test = pd.read_csv(r'data\TextClassification\IMDB\test.csv')

In [5]:
def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

In [6]:
#Tockenize and removing stopwords,punctuations and other irrelevant characters  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing as mp

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]
    return " ".join(words)


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [7]:
df_train['text'] = df_train['text'].apply(remove_stopwords)

In [8]:
df_test['text']=df_test['text'].apply(remove_stopwords)

In [9]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [12]:
# Set the format for PyTorch
train_dataset = train_dataset.rename_column('label', 'labels')
test_dataset = test_dataset.rename_column('label', 'labels')
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [13]:
training_args = TrainingArguments(
    output_dir=r'logs/OtherModels/bert_ag_results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    overwrite_output_dir=True,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="no"
)

# Load the metrics
import evaluate;
accuracy_metric = evaluate.load('accuracy', trust_remote_code=True)
precision_metric = evaluate.load('precision', trust_remote_code=True)
recall_metric = evaluate.load('recall', trust_remote_code=True)

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Sat May 18 18:36:55 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--precision\4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Sat May 18 18:37:00 2024) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\fardin\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--recall\e40e6e98d18ff3f210f4d0b26fa721bfaa80704b1fdf890fa551cfabf94fc185 (last modified on Sat May 18 18:37:06 2024) since it couldn't be found locally

In [15]:
train_dataset[0]['input_ids'].shape

torch.Size([512])

In [22]:
torch.cuda.empty_cache()
import gc
gc.collect()

1572

In [24]:
# Train the model
# trainer.train(resume_from_checkpoint=r"logs/OtherModels/bert_ag_results/last_epoch")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

  0%|          | 0/12500 [00:00<?, ?it/s]

{'loss': 0.4623, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.2}
{'loss': 0.3371, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.4}
{'loss': 0.3, 'learning_rate': 1.76e-05, 'epoch': 0.6}
{'loss': 0.2912, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.8}
{'loss': 0.2928, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


  0%|          | 0/2500 [00:00<?, ?it/s]

{'eval_loss': 0.21821464598178864, 'eval_accuracy': 0.91468, 'eval_precision': 0.9152157306016322, 'eval_recall': 0.9146799999999999, 'eval_runtime': 248.5605, 'eval_samples_per_second': 100.579, 'eval_steps_per_second': 10.058, 'epoch': 1.0}
{'loss': 0.2043, 'learning_rate': 1.5200000000000002e-05, 'epoch': 1.2}
{'loss': 0.1799, 'learning_rate': 1.4400000000000001e-05, 'epoch': 1.4}
{'loss': 0.205, 'learning_rate': 1.3600000000000002e-05, 'epoch': 1.6}
{'loss': 0.2285, 'learning_rate': 1.2800000000000001e-05, 'epoch': 1.8}
{'loss': 0.2084, 'learning_rate': 1.2e-05, 'epoch': 2.0}


  0%|          | 0/2500 [00:00<?, ?it/s]

...

{'eval_loss': 0.4710310399532318, 'eval_accuracy': 0.92092, 'eval_precision': 0.920960759778142, 'eval_recall': 0.92092, 'eval_runtime': 239.6525, 'eval_samples_per_second': 104.318, 'eval_steps_per_second': 10.432, 'epoch': 4.0}
{'loss': 0.0233, 'learning_rate': 3.2000000000000003e-06, 'epoch': 4.2}
{'loss': 0.027, 'learning_rate': 2.4000000000000003e-06, 'epoch': 4.4}
{'loss': 0.0241, 'learning_rate': 1.6000000000000001e-06, 'epoch': 4.6}
{'loss': 0.0159, 'learning_rate': 8.000000000000001e-07, 'epoch': 4.8}
{'loss': 0.0276, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/2500 [00:00<?, ?it/s]

{'eval_loss': 0.527237594127655, 'eval_accuracy': 0.92084, 'eval_precision': 0.9209715883094056, 'eval_recall': 0.92084, 'eval_runtime': 239.4546, 'eval_samples_per_second': 104.404, 'eval_steps_per_second': 10.44, 'epoch': 5.0}
{'train_runtime': 4885.8115, 'train_samples_per_second': 25.584, 'train_steps_per_second': 2.558, 'train_loss': 0.1451786709213257, 'epoch': 5.0}


  0%|          | 0/2500 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.527237594127655, 'eval_accuracy': 0.92084, 'eval_precision': 0.9209715883094056, 'eval_recall': 0.92084, 'eval_runtime': 239.3086, 'eval_samples_per_second': 104.468, 'eval_steps_per_second': 10.447, 'epoch': 5.0}


In [19]:
trainer.save_model(r"logs/OtherModels/bert_ag_results/last_epoch")

In [ ]:
Evaluation results: {'eval_loss': 0.527237594127655, 'eval_accuracy': 0.92084, 'eval_precision': 0.9209715883094056, 'eval_recall': 0.92084, 'eval_runtime': 239.3086, 'eval_samples_per_second': 104.468, 'eval_steps_per_second': 10.447, 'epoch': 5.0}
